In [ ]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-572eeedf-d8b6-2c7d-115e-b0b7d831c42f)


# Installs

In [ ]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

clear_output()

CPU times: user 581 ms, sys: 89.8 ms, total: 671 ms
Wall time: 1min 20s


# Imports

In [ ]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline



CPU times: user 8.55 s, sys: 1.12 s, total: 9.67 s
Wall time: 12.7 s


In [ ]:
modules = [('langchain', langchain), ('torch', torch), ('transformers', transformers)]

for name, module in modules:
    print(f'{name}:', module.__version__)


langchain: 0.1.10
torch: 2.1.0+cu121
transformers: 4.38.2


In [ ]:
sorted(glob.glob('/content/sodapdf-converted.pdf'))

['/content/sodapdf-converted.pdf']

# CFG

- CFG class enables easy and organized experimentation

In [ ]:
class CFG:
    class LLMs:
        model_name = 'llama2-13b-chat'  # wizardlm, llama2-7b-chat, llama2-13b-chat, mistral-7B
        temperature = 0
        top_p = 0.95
        repetition_penalty = 1.15

    class Splitting:
        chunk_size = 800
        overlap = 0

    class Embeddings:
        model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

    class SimilarPassages:
        k = 3

    class Paths:
        PDFs = '/content/'
        Embeddings = '/content/transformer'
        PersistDirectory = './pubmed-vectordb'
print(CFG.Paths.Embeddings)


/content/transformer


In [ ]:
print(CFG.Paths.Embeddings)
  # Should output '/content/transformer'


/content/transformer


# Define model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_model(model=CFG.LLMs.model_name):
    print(f'\nDownloading model: {model}\n\n')

    model_configs = {
        'wizardlm': {
            'repo': 'TheBloke/wizardLM-7B-HF',
            'max_len': 1024,
            'use_fast': False,  # Default is False if not specified
        },
        'llama2-7b-chat': {
            'repo': 'daryl149/llama-2-7b-chat-hf',
            'max_len': 2048,
            'use_fast': True,
        },
        'llama2-13b-chat': {
            'repo': 'daryl149/llama-2-13b-chat-hf',
            'max_len': 2048,  # or 8192 if needed
            'use_fast': True,
        },
        'mistral-7B': {
            'repo': 'mistralai/Mistral-7B-v0.1',
            'max_len': 1024,
            'use_fast': False,
        },
    }

    if model not in model_configs:
        print("Not implemented model (tokenizer and backbone)")
        return None, None, None

    config = model_configs[model]
    tokenizer = AutoTokenizer.from_pretrained(config['repo'], use_fast=config.get('use_fast', False))

    model = AutoModelForCausalLM.from_pretrained(
        config['repo'],
        load_in_4bit=True,
        device_map='auto',
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=config.get('use_fast', False)
    )

    return tokenizer, model, config['max_len']


In [ ]:
%%time


model_info = get_model(model=CFG.LLMs.model_name)

# Unpack the returned values with a check to ensure they're not None
if model_info[0] is not None and model_info[1] is not None and model_info[2] is not None:
    tokenizer, model, max_len = model_info
else:
    print("Failed to load the model. Please check the model name and configurations.")



tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

CPU times: user 43.6 s, sys: 1min 1s, total: 1min 44s
Wall time: 5min 35s


In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    

In [ ]:
def print_accelerate_device_mapping(model):
    if hasattr(model, 'hf_device_map'):
        print("Accelerate Device Mapping:")
        for key, value in model.hf_device_map.items():
            print(f"{key}: {value}")
    else:
        print("Model does not have an attribute 'hf_device_map'. Ensure it's prepared with Accelerate.")


print_accelerate_device_mapping(model)


Accelerate Device Mapping:
: 0


#  pipeline

- Hugging Face pipeline

In [ ]:
class TextGenerationPipelineConfig:
    """
    Configuration for setting up a HuggingFace text generation pipeline.
    This makes it easier to manage and adjust configurations.
    """
    def __init__(self, model, tokenizer, max_length, temperature, top_p, repetition_penalty):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.temperature = temperature
        self.top_p = top_p
        self.repetition_penalty = repetition_penalty

def create_text_generation_pipeline(config):
    """
    Creates and returns a HuggingFace text generation pipeline based on the provided configuration.

    Args:
        config: An instance of TextGenerationPipelineConfig containing all necessary configurations.

    Returns:
        A HuggingFace pipeline object for text generation.
    """
    return pipeline(
        task="text-generation",
        model=config.model,
        tokenizer=config.tokenizer,
        pad_token_id=config.tokenizer.eos_token_id,
        max_length=config.max_length,
        temperature=config.temperature,
        top_p=config.top_p,
        repetition_penalty=config.repetition_penalty
    )

#CFG is a predefined object with necessary attributes
config = TextGenerationPipelineConfig(
    model=model,
    tokenizer=tokenizer,
    max_length=max_len,
    temperature=CFG.LLMs.temperature,
    top_p=CFG.LLMs.top_p,
    repetition_penalty=CFG.LLMs.repetition_penalty
)

pipe = create_text_generation_pipeline(config)
llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x78ae0cf70910>)

In [ ]:
%%time

query = "what is the role of emotional intelligence"
llm.invoke(query)

CPU times: user 1min 12s, sys: 497 ms, total: 1min 13s
Wall time: 1min 16s


" in leadership?\nEmotional intelligence (EI) plays a crucial role in effective leadership. Here are some reasons why:\n1. Self-awareness: Leaders with high EI have a better understanding of their own emotions, strengths, and weaknesses. This self-awareness helps them make informed decisions, communicate effectively, and build strong relationships with their team members.\n2. Empathy: Emotionally intelligent leaders can empathize with their team members, understand their perspectives, and create a positive work environment. They can also manage conflicts more effectively by recognizing the emotions behind the conflict and addressing them constructively.\n3. Communication skills: Leaders with high EI communicate more effectively, as they are better able to read the emotions of others and respond appropriately. They can also use their emotional intelligence to motivate and inspire their teams.\n4. Conflict resolution: Emotionally intelligent leaders are better at resolving conflicts, as 

#  Langchain

- Multiple document retriever with LangChain

In [ ]:
CFG.LLMs.model_name

'llama2-13b-chat'

# Loader

In [ ]:
import time

def load_pdf_documents(directory_path, glob_pattern="./*.pdf", use_multithreading=True, show_progress=True):
    """
    Loads PDF documents from a specified directory using PyPDFLoader.

    Args:
        directory_path (str): Path to the directory containing PDF files.
        glob_pattern (str): Pattern to match files in the directory. Defaults to "./*.pdf".
        use_multithreading (bool): Whether to use multithreading for loading documents. Defaults to True.
        show_progress (bool): Whether to show progress during loading. Defaults to True.

    Returns:
        A list of loaded PDF documents.
    """
    start_time = time.time()

    loader = DirectoryLoader(
        directory_path,
        glob=glob_pattern,
        loader_cls=PyPDFLoader,
        show_progress=show_progress,
        use_multithreading=use_multithreading
    )
    documents = loader.load()

    end_time = time.time()
    print(f"Completed loading documents in {end_time - start_time:.2f} seconds.")

    return documents

# Use the function to load PDF documents
documents = load_pdf_documents(CFG.Paths.PDFs)


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]

Completed loading documents in 1.80 seconds.


In [ ]:
 print(f'We have {len(documents)} pages in total')

We have 141 pages in total


In [ ]:
def get_document_page_content(documents, document_index):
    """
    Safely retrieves the page content of a specified document from a list of documents.

    Args:
        documents (list): A list of document objects.
        document_index (int): The index of the document from which to retrieve the page content.

    Returns:
        The page content of the specified document if available; otherwise, returns a descriptive error message.
    """
    try:
        # Ensure the document index is within the valid range
        if document_index < 0 or document_index >= len(documents):
            return "Error: Document index is out of range."

        # Attempt to access the document's page content
        page_content = documents[document_index].page_content
        return page_content
    except AttributeError:
        return "Error: Document does not have 'page_content' attribute."
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


document_index = 8  # Adjust as needed
page_content = get_document_page_content(documents, document_index)
print(page_content)


PMID: 33981990  
  
Conflict of interest statement: The authors declare that the research was   
conducted in the absence of any commercial or financial relationships that could   
be construed as a potential conflict of interest.  
  
  
12. Front Neurosci. 2022 Oct 6;16:1024316. doi: 10.3389/fnins.2022.1024316.   
eCollection 2022.  
  
Cognitive psychology-based artificial intelligence review.  
  
Zhao J(1), Wu M(1), Zhou L(1), Wang X(1), Jia J(2)(3).  
  
Author information:  
(1)School of Information Science and Technology, Northwest University, Xi'an,   
China.  
(2)Medical Big Data Research Center, Northwest University, Xi'an, China.  
(3)School of Mathematics, Northwest University, Xi'an, China.  
  
Most of the current development of artificial intelligence is based on brain   
cognition, however, this replication of biology cannot simulate the subjective   
emotional and mental state changes of human beings. Due to the imperfections of   
existing artificial intelligence, th

In [ ]:
# documents[8].page_content

"PMID: 33981990  \n  \nConflict of interest statement: The authors declare that the research was   \nconducted in the absence of any commercial or financial relationships that could   \nbe construed as a potential conflict of interest.  \n  \n  \n12. Front Neurosci. 2022 Oct 6;16:1024316. doi: 10.3389/fnins.2022.1024316.   \neCollection 2022.  \n  \nCognitive psychology-based artificial intelligence review.  \n  \nZhao J(1), Wu M(1), Zhou L(1), Wang X(1), Jia J(2)(3).  \n  \nAuthor information:  \n(1)School of Information Science and Technology, Northwest University, Xi'an,   \nChina.  \n(2)Medical Big Data Research Center, Northwest University, Xi'an, China.  \n(3)School of Mathematics, Northwest University, Xi'an, China.  \n  \nMost of the current development of artificial intelligence is based on brain   \ncognition, however, this replication of biology cannot simulate the subjective   \nemotional and mental state changes of human beings. Due to the imperfections of   \nexisting art

# Splitter

In [ ]:
def split_documents_into_chunks(documents, chunk_size, chunk_overlap):
    """
    Splits documents into smaller text chunks based on specified chunk size and overlap.

    Args:
        documents (list): A list of documents to be split.
        chunk_size (int): The size of each chunk in characters.
        chunk_overlap (int): The number of characters to overlap between chunks.

    Returns:
        list: A list of text chunks obtained from splitting the documents.
    """
    # Initialize the text splitter with the given chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    # Split the documents
    texts = text_splitter.split_documents(documents)

    return texts

# Use the function to split documents
CFG = {
    "split_chunk_size": 1000,  # Example chunk size
    "split_overlap": 100       # Example overlap size
}

texts = split_documents_into_chunks(documents, CFG["split_chunk_size"], CFG["split_overlap"])

print(f'We have created {len(texts)} chunks from {len(documents)} pages')


We have created 492 chunks from 141 pages


# Create Embeddings


In [ ]:

# Assuming CFG class definition as provided
if not os.path.exists(os.path.join(CFG.Paths.Embeddings, 'faiss_index.faiss')):
    embeddings = HuggingFaceInstructEmbeddings(
      model_name=CFG.Embeddings.model_repo,
      model_kwargs={"device": "cuda"}
    )

    vectordb = FAISS.from_documents(
      documents=texts,
      embedding=embeddings
    )

    vectordb.save_local(CFG.Paths.Embeddings)



.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


# Load vector database


In [ ]:


def download_embeddings_model(model_repo, device="cuda"):
    """
    Downloads an embeddings model from HuggingFace.

    Args:
        model_repo (str): The repository name of the model on HuggingFace.
        device (str): The device to run the model on. Defaults to 'cuda'.

    Returns:
        The downloaded model.
    """
    try:
        embeddings = HuggingFaceInstructEmbeddings(
            model_name=model_repo,
            model_kwargs={"device": device}
        )
        print(f"Model {model_repo} downloaded successfully.")
        return embeddings
    except Exception as e:
        print(f"Error downloading model {model_repo}: {e}")
        return None

def load_vector_database(embeddings_path, embeddings):
    """
    Loads a local FAISS vector database.

    Args:
        embeddings_path (str): The path to the FAISS index file.
        embeddings: The embeddings model used for vectorization.

    Returns:
        The loaded FAISS database.
    """
    try:
        vectordb = FAISS.load_local(
            embeddings_path,
            embeddings
        )
        print(f"Vector database loaded from {embeddings_path}.")
        return vectordb
    except Exception as e:
        print(f"Error loading vector database from {embeddings_path}: {e}")
        return None


embeddings_model_repo = CFG.Embeddings.model_repo  
embeddings = download_embeddings_model(embeddings_model_repo)
if embeddings:
    vectordb = load_vector_database(CFG.Paths.Embeddings, embeddings)
    if vectordb:
        clear_output(wait=True)
        print("Embeddings model and vector database are ready.")
else:
    print("Failed to set up embeddings model and vector database.")


Embeddings model and vector database are ready.


In [ ]:
### test if vector DB was loaded correctly
vectordb.similarity_search('AI')

[Document(page_content='Francisco, San Francisco, California. Electronic address:   \nmarcelle.cedars@ucsf.edu.  \n  \nArtificial intelligence, at a simple level, involves the use of a computer that   \ncan perform "human" functions: learning from experience, adjusting to new   \ninputs, and simulating human intelligence performing human tasks. This Views and   \nReviews brings together a diverse group of investigators to evaluate artificial', metadata={'source': '/content/sodapdf-converted.pdf', 'page': 86}),
 Document(page_content='part of a system-wide approach to describe other complex open systems. The paper   \nsuggests three layers and five levels of the AGI model. Two levels (entropy and   \nprocess) are at the technological layer of AI functioning, two more levels   \n(social and linguistic ones) are at the relationship layer responsible for the   \nbehavior of AI, and, finally, the uppermost level (actualization) supposes   \ngeneral intelligence. All the components of each u

# Prompt Template

- Custom prompt

In [ ]:
prompt_template = """

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [ ]:
# llm_chain = LLMChain(prompt=PROMPT, llm=llm)
# llm_chain

# Retriever chain

- Retriever to retrieve relevant passages
- Chain to answer questions

In [ ]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.SimilarPassages.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [ ]:
### testing MMR search
question = "What is AI?"
vectordb.max_marginal_relevance_search(question, k = CFG.SimilarPassages.k)

[Document(page_content='Francisco, San Francisco, California. Electronic address:   \nmarcelle.cedars@ucsf.edu.  \n  \nArtificial intelligence, at a simple level, involves the use of a computer that   \ncan perform "human" functions: learning from experience, adjusting to new   \ninputs, and simulating human intelligence performing human tasks. This Views and   \nReviews brings together a diverse group of investigators to evaluate artificial', metadata={'source': '/content/sodapdf-converted.pdf', 'page': 86}),
 Document(page_content='2023 May 17.  \n  \nLay representations of artificial intelligence and autonomous military machines.  \n  \nWagner W, Viidalepp A(1), Idoiaga-Mondragon N(2), Talves K(3), Lillemäe E,   \nPekarev J, Otsus M(4).  \n  \nAuthor information:  \n(1)University of Tartu, Estonia.  \n(2)University of the Basque Country, Spain.  \n(3)Estonian Ministry of Defence, Estonia.  \n(4)Estonian Military Academy, Estonia.  \n  \nThis study is about how lay persons perceive a

In [ ]:
### testing similarity search
question = "What is emotional intelligence?"
vectordb.similarity_search(question, k = CFG.SimilarPassages.k)

[Document(page_content='articles.  \nRESULTS: Three categories were identified: Constructs of emotional intelligence;   \nemotional intelligence curricula components; and strategies for emotional   \nintelligence education.  \nCONCLUSIONS: A wide range of emotional intelligence constructs were found, with   \na predominance of trait-based constructs. A variety of strategies to enhance', metadata={'source': '/content/sodapdf-converted.pdf', 'page': 48}),
 Document(page_content='life has made it the subject of expert study. EI is the rudder for feeling,   \nthinking, learning, problem-solving, and decision-making. In this article, we   \npresent an emotional\x00cognitive based approach to the process of gaining   \nemotional intelligence and thus, we suggest a nine-layer pyramid of emotional   \nintelligence and the gradual development to reach the top of EI.  \n  \nDOI: 10.3390/bs8050045  \nPMCID: PMC5981239  \nPMID: 29724021  \n  \nConflict of interest statement: The authors declare no

# Post-process outputs

- Format llm response
- Cite sources (PDFs)
- Change `width` parameter to format the output

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=70):
    """
    Wraps text to a specified width while preserving original newline characters.

    Args:
        text (str): The input text to be wrapped.
        width (int): The maximum width of the text in characters.

    Returns:
        str: The wrapped text with original newline characters preserved.
    """
    # Wrap each line individually to preserve newline characters.
    wrapped_lines = [textwrap.fill(line, width=width) for line in text.split('\n')]

    # Join the individually wrapped lines back together.
    return '\n'.join(wrapped_lines)

def process_llm_response(llm_response):
    """
    Processes the LLM response by wrapping the text and appending source information.

    Args:
        llm_response (dict): The LLM response, expected to contain 'result' and 'source_documents'.

    Returns:
        str: The processed text with sources appended.
    """
    # Wrap the LLM response text.
    ans = wrap_text_preserve_newlines(llm_response['result'])

    # Format source information from the LLM response.
    sources_used = ' \n'.join(
        f"{source.metadata['source'].split('/')[-1][:-4]} - page: {str(source.metadata['page'])}"
        for source in llm_response.get('source_documents', [])
    )

    # Append formatted sources information to the wrapped text.
    ans += '\n\nSources: \n' + sources_used
    return ans


In [ ]:
import time

def llm_ans(query):
    """
    Executes a query using a Large Language Model (LLM), processes the response,
    measures the time taken for the operation, and appends this information to the response.

    Args:
        query (str): The query to be executed by the LLM.

    Returns:
        str: The processed response from the LLM appended with the time elapsed for the query execution.
    """
    start = time.time()

    try:
        llm_response = qa_chain.invoke(query)
        ans = process_llm_response(llm_response)
    except Exception as e:
        ans = f"An error occurred while processing the query: {e}"

    end = time.time()
    time_elapsed = round(end - start)  # Time elapsed in seconds, rounded to the nearest whole number
    time_elapsed_str = f"\n\nTime elapsed: {time_elapsed} s"

    return ans + time_elapsed_str


# Ask questions

- Question Answering from multiple documents
- Invoke QA Chain

In [ ]:
CFG.LLMs.model_name

'llama2-13b-chat'

In [ ]:
# Debugging
print(f"qa_chain: {qa_chain}")



qa_chain: combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nDon't try to make up an answer, if you don't know just say that you don't know.\nAnswer in the same language the question was asked.\nUse only the following pieces of context to answer the question at the end.\n\n{context}\n\nQuestion: {question}\nAnswer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x78ae0cf70910>)), document_variable_name='context') return_source_documents=True retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78ae0cf71780>, search_kwargs={'k': 3, 'search_type': 'similarity'})


In [ ]:
query = "In which fields Artificial Intelligence is Playing important role?"
print(llm_ans(query))

 Artificial Intelligence is playing an important role in various
fields including but not limited to:

* Human-versus Artificial Intelligence
* System-wide approach to describe other complex open systems
* Technological layer of AI functioning
* Relationship layer responsible for the behavior of AI
* Actualization of intelligence

Note: The above answer is based on the given text and does not include
any external knowledge or research.

Sources: 
sodapdf-converted - page: 86 
sodapdf-converted - page: 7 
sodapdf-converted - page: 133

Time elapsed: 20 s


In [ ]:
query = "what is emotional intelligence?"
print(llm_ans(query))

 Emotional intelligence refers to the ability to recognize and
understand emotions in oneself and others, and to use this awareness
to guide thought and behavior. It involves the ability to regulate
one's own emotions, as well as the ability to respond appropriately to
the emotions of others.

Context: The information provided comes from three different sources:
an article discussing emotional intelligence and its various
constructs, another article describing the development and validation
of a new scale to measure emotional intelligence, and a third article
discussing the concept of emotional intelligence as a developing
function.

Sources: 
sodapdf-converted - page: 48 
sodapdf-converted - page: 126 
sodapdf-converted - page: 126

Time elapsed: 24 s


In [ ]:
query = "What is Mandevillian intelligence?"
print(llm_ans(query))

 Based on the provided text, there is no direct mention of
"Mandevillian intelligence." However, based on the context of the
text, it appears to be a specific form of collective intelligence
where individual cognitive vices play a positive functional role in
yielding collective forms of cognitive success.

Sources: 
sodapdf-converted - page: 5 
sodapdf-converted - page: 122 
sodapdf-converted - page: 7

Time elapsed: 15 s


In [ ]:
query = "What are 5 examples of current development in artificial intelligence is based on brain cognition"
print(llm_ans(query))

 Based on the text provided, here are five examples of current
developments in artificial intelligence that are based on brain
cognition:

1. Computational models of cognitive processes, such as those used in
artificial neural networks.
2. Reverse engineering of the best existing implementation of
intelligence, as suggested by the quote "We should reverse-engineer
the best existing implementation of intelligence."
3. The use of brain imaging techniques like fMRI to study the neural
basis of cognition and inform AI development.
4. The integration of insights from neuroscience into AI systems, as
seen in the quote "Progresses in brain science can contribute to
future developments in artificial intelligence."
5. The application of machine learning algorithms to analyze large
datasets and learn from experience, as mentioned in the quote
"Learning from experience, adjusting to new inputs, and simulating
human intelligence performing human tasks."

Sources: 
sodapdf-converted - page: 68 
sod

# Gradio Chat UI

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
! pip install --upgrade gradio -qq
clear_output()

In [ ]:
import gradio as gr

In [ ]:
# import gradio as gr
print(gr.__version__)

4.19.2


In [ ]:
def predict(message, history):
    # output = message # debug mode

    output = str(llm_ans(message)).replace("\n", "<br/>")
    return output

demo = gr.ChatInterface(
    predict,
    title = f' Open-Source LLM ({CFG.LLMs.model_name}) for PubMed Question Answering'
)
# Launch the app
demo.queue()
demo.launch()
#https://2121518c46ba03d354.gradio.live/

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e269ee3cd829a1c415.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
